In [1]:
import pandas as pd
from math import floor
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
import requests
from src.paths import LOCAL_RAW_DATA_PATH

c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
ts = pd.date_range(start='2019-06-24 00:48:00', end='2021-06-12 20:53:00', freq='min')

In [3]:
def build_url(from_:str, to:str, since:str, until:str, period:str='M1', limit:int=1000):
    endpoint = 'https://api.hitbtc.com/api/3/public/price/history'
    url = f'{endpoint}?from={from_}&to={to}&period={period}&since={since}&until={until}&limit={limit}'
    return url

In [4]:
step = 1000
slices = round(ts.shape[0] / step)
for i in tqdm(range(slices)):
    s_idx = i*step
    f_idx = min(s + step, ts.shape[0])
    ts_sliced = ts[s_idx:f_idx]
    since = str(ts_sliced.min())
    until = str(ts_sliced.max())
    url = build_url(from_='BTC', to='USDT', since=since, until=until)
    if i == 1:
        break
    pass
url

  0%|          | 0/1037 [00:00<?, ?it/s]

NameError: name 's' is not defined

In [5]:
df = pd.concat([
    pd.read_csv(
        fr'C:\Users\muril\Documents\GitHub\crypto-prophet\data\raw\Bitstamp_BTCUSD_{x}_minute.csv',
        skiprows=1)
    for x in range(2019,2022)
])

In [6]:
df['date'] = pd.to_datetime(df.date)

In [7]:
df = df.sort_values('date')

In [8]:
mask = (df.date >= '2019-06-24 00:48:00') & (df.date < '2021-06-12 20:54:00')
df = df[mask]

In [9]:
df = df.rename({'Volume BTC': 'volume'}, axis=1)
df

,unix,date,symbol,open,high,low,close,volume,Volume USD
274991,1561337280,2019-06-24 00:48:00,BTC/USD,10666.67,10677.33,10665.46,10671.75,5.676160,60574.560800
274990,1561337340,2019-06-24 00:49:00,BTC/USD,10680.00,10682.67,10669.20,10671.39,0.231995,2475.705175
274989,1561337400,2019-06-24 00:50:00,BTC/USD,10672.53,10685.00,10672.53,10682.89,9.755684,104218.895308
274988,1561337460,2019-06-24 00:51:00,BTC/USD,10683.58,10688.26,10680.35,10687.01,6.301186,67340.835977
274987,1561337520,2019-06-24 00:52:00,BTC/USD,10688.26,10699.73,10687.01,10697.00,7.774960,83168.749259
...,...,...,...,...,...,...,...,...,...
177712,1623530940,2021-06-12 20:49:00,BTC/USD,36080.44,36102.56,36065.55,36072.88,0.453399,16355.417098
177711,1623531000,2021-06-12 20:50:00,BTC/USD,36082.40,36132.26,36077.87,36109.25,4.280521,154566.420613
177710,1623531060,2021-06-12 20:51:00,BTC/USD,36128.38,36141.67,36103.70,36123.15,0.207626,7500.100807
177709,1623531120,2021-06-12 20:52:00,BTC/USD,36123.15,36123.15,36070.88,36073.20,0.073485,2650.854613


In [13]:
from src.helpers import get_valorization
for p in [+24, +12, +6, +2, +1, -1, -2, -6, -12, -24, -24*2, -24*7, -24*15, -25*30]:
    val = get_valorization(df, periods=p)
    df[val.name] = val

In [14]:
df.columns

Index(['unix', 'date', 'symbol', 'open', 'high', 'low', 'close', 'volume',
       'Volume USD', 'close_change_previous12periods_perc',
       'close_change_previous6periods_perc',
       'close_change_previous2periods_perc',
       'close_change_previous1periods_perc', 'close_change_1periods_perc',
       'close_change_2periods_perc', 'close_change_6periods_perc',
       'close_change_12periods_perc', 'close_change_24periods_perc',
       'close_change_48periods_perc', 'close_change_168periods_perc',
       'close_change_360periods_perc', 'close_change_750periods_perc',
       'close_change_previous24periods_perc'],
      dtype='object')

In [15]:
import numpy as np
stats_cols = []
for ma in (120, 720):
    column_name = f"ma_{ma}_periods"
    stats_cols.append(column_name)

    df[column_name] = df['close'].rolling(ma).mean()
    

df['ma_delta'] = df.apply(
        lambda row: row[stats_cols[0]] - row[stats_cols[1]]
        if not np.isnan(row[stats_cols[1]])
        else np.nan, axis=1)

df['ma_delta_norm'] = df.apply(
        lambda row: row['ma_delta'] / row['close'], axis=1)

In [ ]:
x =    pd.read_csv(
        fr'C:\Users\muril\Documents\GitHub\crypto-prophet\data\raw\btc-usd.csv',
        )


In [ ]:
df[x.columns].to_csv(LOCAL_RAW_DATA_PATH / 'btc-usd-minute.csv')